# hw4 - churn prediction of bank customers

In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split

from category_encoders import GLMMEncoder
from category_encoders import cat_boost

from sklearn.feature_selection import SequentialFeatureSelector


from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

db = pd.read_csv('train.csv').drop(labels=['RowNumber','CustomerId','Surname'],axis =1)
five_features = ['Geography','Gender', 'Age', 'NumOfProducts', 'IsActiveMember','HasCrCard']

In [18]:
catcols = db.select_dtypes(exclude='number').columns
glmm = GLMMEncoder(
    binomial_target = True,
    cols = catcols,
)

xgb = XGBClassifier()

para_xgb = {
    'n_estimators':[100,200,300,400,500,600,700],
    'max_depth':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],
    'learning_rate':[0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],
    'min_child_weight':[1,2,3,4,5,6]
}
xgb_RS = GridSearchCV(
    estimator = xgb,
    param_grid = para_xgb,
)

sfs = SequentialFeatureSelector(
    xgb_RS,
    n_features_to_select = 5,
    direction = 'backward'
)

In [19]:
glmm.fit(db.drop(['Exited'],axis=1),db['Exited'])

GLMMEncoder(binomial_target=True,
            cols=Index(['Geography', 'Gender'], dtype='object'))

In [20]:
X_train, X_test, y_train, y_test = train_test_split( 
    db.drop(['Exited'],axis=1),
    db['Exited'],
    test_size = 0.8,
    random_state=1001
)

In [21]:
X_train = glmm.transform(X_train)
X_test = glmm.transform(X_test)

In [22]:
#sfs.fit(X_train,y_train)

In [23]:
X_train = X_train.loc[:,five_features]
X_test = X_test.loc[:,five_features]

In [24]:
xgb_RS.fit(X_train,y_train)

GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_b...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             param_grid={'learning_rate': [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6,
                                           0.7, 0.8],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15],
                         'min_child_weight': [1, 2, 3, 4, 5, 6],
                         'n_estimators': [100, 200, 300, 400, 500, 600, 700]})

In [25]:
xgb_RS.score(X_test,y_test)

0.858125

In [26]:
raw = pd.read_csv('test.csv').drop(labels=['RowNumber','CustomerId','Surname'],axis = 1)
form = pd.read_csv('upload_test.csv')

In [27]:
raw = glmm.transform(raw)
raw = raw.loc[:,five_features]

In [28]:
ans = xgb_RS.predict(raw)
form['Exited'] = ans
pd.DataFrame(ans,columns=['Exited']).value_counts()

Exited
0         1744
1          256
dtype: int64

In [29]:
form.to_csv('upload_test.csv')